In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-04-09"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
9319,2024-04-09,Brasil Nbb,20:00,São José,São Paulo,1.78,1.94,158.5,1.83,1.87,-1.5,1.92,2.10,IZsZwyog,0.561798,0.515464,0.546448,0.534759,0.077262,0.2,0.8,NaN,NaN,159.078,41.240844,0.259249,192.512,94.612959,0.491465,117.994,197.444,100.86,354.38,0.0,0.0,0.0,0.0,0.060826,0.015289,0.063323,-1.14,-0.228,-3.421053,0.671129,0.7,0.028871,2.55,0.510,1.843137,0.571000,0.6,0.029000
9320,2024-04-09,Eua Nba,20:30,Atlanta Hawks,Miami Heat,2.27,1.68,222.5,1.80,1.95,1.5,2.05,1.53,f50ybM0J,0.440529,0.595238,0.555556,0.512821,0.035767,0.2,0.8,NaN,NaN,277.622,139.828930,0.503667,242.654,30.142336,0.124219,206.006,283.596,235.20,233.45,0.0,0.0,0.0,0.0,0.211237,0.056569,0.205416,2.12,0.424,2.995283,0.554532,0.7,0.145468,2.69,0.538,1.263941,0.438663,0.7,0.261337
9321,2024-04-09,Eua Nba,20:30,Milwaukee Bucks,Boston Celtics,2.11,1.78,227.5,1.93,1.93,1.5,2.01,1.63,ShsUbp1J,0.473934,0.561798,0.518135,0.518135,0.035731,0.4,0.4,NaN,NaN,189.418,37.879512,0.199978,166.054,20.487210,0.123377,180.164,166.998,205.32,156.00,0.0,0.0,0.0,0.0,0.119972,0.000000,0.147638,1.35,0.270,4.111111,0.654645,0.7,0.045355,0.53,0.106,7.358491,0.669284,0.8,0.130716
9322,2024-04-09,Eua Nba,21:00,Chicago Bulls,New York Knicks,2.41,1.61,212.5,1.80,1.95,2.5,2.05,1.40,6qtYc4GP,0.414938,0.621118,0.555556,0.512821,0.036056,0.0,0.2,NaN,NaN,204.706,59.682483,0.291552,191.248,11.846194,0.061942,202.350,218.498,208.44,193.00,0.0,0.0,0.0,0.0,0.281436,0.056569,0.266446,-0.80,-0.160,-8.812500,0.481091,0.4,-0.081091,1.42,0.284,2.147887,0.586681,0.7,0.113319
9323,2024-04-09,Eua Nba,21:00,Houston Rockets,Orlando Magic,2.20,1.72,212.5,1.80,1.95,1.5,2.00,1.57,OOSagOOt,0.454545,0.581395,0.555556,0.512821,0.035941,0.6,0.4,NaN,NaN,192.320,28.463129,0.147999,180.824,31.844512,0.176108,230.856,201.818,200.72,142.82,0.0,0.0,0.0,0.0,0.173169,0.056569,0.170339,-1.08,-0.216,-5.555556,0.501163,0.4,-0.101163,-0.74,-0.148,-4.864865,0.501624,0.4,-0.101624
9324,2024-04-09,Eua Nba,21:00,Memphis Grizzlies,San Antonio Spurs,2.31,1.66,214.5,1.80,1.95,2.5,2.00,1.46,xSW3hrvm,0.432900,0.602410,0.555556,0.512821,0.035310,0.4,0.4,NaN,NaN,307.876,162.584256,0.528084,473.766,217.166758,0.458384,212.396,523.610,292.64,681.12,0.0,1.0,0.0,0.0,0.231546,0.056569,0.220715,-3.24,-0.648,-2.021605,0.449181,0.3,-0.149181,-1.32,-0.264,-2.500000,0.243431,0.1,-0.143431
9325,2024-04-09,Eua Nba,21:00,Oklahoma City Thunder,Sacramento Kings,1.63,2.36,227.5,1.80,1.95,-4.5,2.00,3.20,bsLCjM8a,0.613497,0.423729,0.555556,0.512821,0.037226,0.6,0.8,NaN,NaN,229.170,36.334895,0.158550,201.690,46.648505,0.231288,189.752,279.048,218.88,256.15,0.0,0.0,0.0,0.0,0.258741,0.056569,0.326357,2.38,0.476,1.323529,0.742119,0.9,0.157881,2.21,0.442,3.076923,0.459630,0.5,0.040370
9326,2024-04-09,Eua Nba,23:00,Los Angeles Lakers,Golden State Warriors,1.78,2.11,232.5,1.93,1.93,-3.5,2.05,2.60,W8YJl0wC,0.561798,0.473934,0.518135,0.518135,0.035731,0.6,0.4,NaN,NaN,204.666,33.124426,0.161846,232.382,52.499336,0.225918,234.294,253.366,178.64,293.62,0.0,0.0,0.0,0.0,0.119972,0.000000,0.167273,2.68,0.536,1.455224,0.590904,0.8,0.209096,1.21,0.242,4.586777,0.460559,0.7,0.239441
9327,2024-04-09,Eua Nba,23:00,Phoenix Suns,Los Angeles Clippers,1.63,2.36,224.5,1.80,1.95,-4.5,2.00,3.20,EXNOmKhI,0.613497,0.423729,0.555556,0.512821,0.037226,0.2,0.2,NaN,NaN,207.338,51.590941,0.248825,195.822,34.775975,0.177590,172.236,213.014,161.02,188.00,0.0,0.0,0.0,0.0,0.258741,0.056569,0.326357,0.45,0.090,7.000000,0.68

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,20:00,Brasil Nbb,São José,São Paulo,158.5,1.83,0.9998,Over
1,21:00,Eua Nba,Oklahoma City Thunder,Sacramento Kings,227.5,1.80,1.0000,Over
2,23:00,Eua Nba,Los Angeles Lakers,Golden State Warriors,232.5,1.93,1.0000,Over
3,23:00,Eua Nba,Phoenix Suns,Los Angeles Clippers,224.5,1.80,1.0000,Over
4,07:00,Coreia Do Sul Kbl,Mobis Phoebus,Suwon KT,173.5,1.87,1.0000,Over
5,12:30,Finlândia Korisliiga,Kataja,UU-Korihait,166.5,1.88,1.0000,Over
6,13:00,Israel Liga Leumit,Maccabi Raanana,Elitzor Netania,163.5,1.80,0.9930,Over
7,15:00,Polônia Liga De Basquete,Slask Wroclaw,Legia,165.5,1.87,1.0000,Over
8,14:00,Rússia Liga Vtb United,Lokomotiv Kuban,Zenit Petersburg,167.5,1.81,1.0000,Over
9,21:00,Porto Rico Bsn,San German,Aguada Santeros,167.5,1.82,0.8530,Over
